! pip install youtube-comment-downloader 

In [3]:
from youtube_comment_downloader import YoutubeCommentDownloader
import pandas as pd
import time 


In [ ]:
#video_id ber korte hobe 
#

In [4]:
video_ids = [
    "K0B2qlb8eVI",
    "5NLf7O6BF6s"
]

downloader = YoutubeCommentDownloader()
all_data = []

for vid in video_ids:
    print(f"Downloading comments for video: {vid}")
    url = f"https://www.youtube.com/watch?v={vid}"

    try:
        comments = downloader.get_comments_from_url(url)
        for comment in comments:
            # Some videos/comments do not include all fields (e.g., likes).
            likes = comment.get("likes", comment.get("votes", 0))
            all_data.append({
                "video_id": vid,
                "comment": comment.get("text", ""),
                "author": comment.get("author", ""),
                "likes": likes,
                "published_at": comment.get("published_at", comment.get("time", ""))
            })
            time.sleep(2)  # Sleep to avoid rate limits
    except Exception as e:
        print(f"Error downloading comments for video {vid}: {e}")

df = pd.DataFrame(all_data)
df.to_csv("youtube_comments.csv", index=False, encoding='utf-8-sig')
print("Comments downloaded and saved to youtube_comments.csv")


Error downloading comments for video K0B2qlb8eVI: 'likes'
Error downloading comments for video 5NLf7O6BF6s: 'likes'
Comments downloaded and saved to youtube_comments.csv
